<a href="https://colab.research.google.com/drive/10FNhJMwKokjIzh9MBnRM3t4Kz9vM1BuC" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FiftyOne Workshop - Agriculture
# Model Evaluation

## 🏆 Learning Objectives
- Use Remote Models for your model evaluation
- Load predictions in your actual daaset
- Use model evaluation Panel for model comparison

## Requirements
### Knowledge
- Understanding of image segmentation.
- Familiarity with deep learning-based annotation tools.
### Installation
This installation process has tested with Python 3.11 and 3.10. Run the following commands to install necessary dependencies:
```bash
pip install fiftyone
python -m pip install --upgrade pip wheel setuptools
pip install geti-sdk
```

In [ ]:
!pip install fiftyone

In [ ]:
import torch

def get_device():
    """Get the appropriate device for model inference."""
    if torch.cuda.is_available():
        return "cuda"
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

DEVICE = get_device()

print(f"Using device: {DEVICE}")

In [ ]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh
from fiftyone.utils.coco import COCODetectionDatasetImporter

import gdown

# Download the coffee dataset from Google Drive

url = "https://drive.google.com/uc?id=1bzAxeSXKe3vHx_Py-VQTc1cOgYpChl8s" # evaluation
gdown.download(url, output="coffee_evaluation_FO.zip", quiet=False)

In [ ]:
!unzip coffee_evaluation_FO.zip

If dataset is already in disk just use the next cell for loading it again.

In [ ]:
import fiftyone as fo

dataset_viz =fo.Dataset.from_dir(
    dataset_dir="/content/coffee_evaluation_FO",
    dataset_type=fo.types.FiftyOneDataset,
    name="coffee_evaluation",
)

In [ ]:
session = fo.launch_app(dataset_viz, auto=False)
session.open_tab()

# ---------------------------------------------------------------------

If you want to run the inference, run the rest of the notebook, following the instructions.

## 1. Loading the Dataset

For education purposes, use this link in Drive for downloading an upgraded dataset with 100+ annotated unique images.

Download the dataset with this [Link](https://drive.google.com/file/d/1aCr00sF2hjLw7hpq3yeXNUvC07TXdQsg/view?usp=sharing)

In [ ]:
import fiftyone as fo


from fiftyone.utils.coco import COCODetectionDatasetImporter


dataset = fo.Dataset.from_dir(
   dataset_type=fo.types.COCODetectionDataset,
   dataset_dir="./colombian_coffee-dataset_1600",
   data_path="images/default",
   labels_path="annotations/instances_default.json",
   label_types="segmentations",
   label_field="categories",
   name="coffee",
   include_id=True,
   overwrite=True
)

view = dataset.shuffle()

## 2. Preparing the models for inference

We are using Intel Geti SDK for runing the inference of the Intel Geti Models. I have download the deplyments of two models I have created 1- Before data quality improvement, 2- After data quality improvement. Please download the deploment folders using these links and unzip them in the same location of this notebook.

Model 1: [Link](https://drive.google.com/file/d/1r_Xi89dTlXt9o2Tj0JQXR2XDuRXHXfQB/view?usp=sharing)

Model 2: [Link](https://drive.google.com/file/d/1fkq-4ouMV2owzbV7MEOrOuOIA_UPaJUP/view?usp=sharing)


In [ ]:
from openvino.runtime import Core

ie = Core()
devices = ie.available_devices

for device in devices:
    device_name = ie.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

After you have the models in your local disk, we can load the models into memory to prepare them for inference. There you can select the device for running the inference, in OpenVINO we have different options. You can setup CPU, GPU, AUTO for auto plugin, and MULTI for runnig the model in multiple devices.

In [ ]:
from geti_sdk.deployment import Deployment

deployment_folder_model_selected_1 = "/home/paula/projects/databricks/sam2/geti_sdk-deployment_57"
deployment_folder_model_selected_2 = "/home/paula/projects/databricks/sam2/geti_sdk-deployment_90"

deployment1 = Deployment.from_folder(deployment_folder_model_selected_1)
deployment2 = Deployment.from_folder(deployment_folder_model_selected_2)

#deployment1.load_inference_models(device="CPU")
deployment2.load_inference_models(device="CPU")


For education purposes we will inspect predictions and annotations coming from my model

In [ ]:
import fiftyone as fo
from PIL import Image as PILImage  # Using PIL's Image
import numpy as np


# Function to process a single sample and inspect the prediction
def inspect_prediction_for_sample(sample):
    # Get the image path from the sample
    image_path = sample.filepath  # Assuming 'filepath' exists in your sample

    # Load the image using PIL
    image_data = PILImage.open(image_path)  # Open image using PIL

    # Convert the PIL image to a numpy array
    image_data = np.array(image_data)

    # Run the inference on the image
    prediction = deployment1.infer(image_data)

    # Print the structure of the prediction object by listing its attributes
    print("Prediction attributes:", dir(prediction))

    # If there are specific attributes, inspect them
    # For example, if there is an attribute like 'segmentation'
    if hasattr(prediction, 'segmentation'):
        print("Segmentation Data:", prediction.segmentation)

    # Check if other useful attributes exist
    if hasattr(prediction, 'labels'):
        print("Labels Data:", prediction.labels)

# Select a single sample from the dataset to test (you can pick the first one for simplicity)
sample = dataset.first()

# Run the function to inspect the prediction for this sample
inspect_prediction_for_sample(sample)


Prediction attributes: ['_GET_only_fields', '__annotations__', '__attrs_attrs__', '__attrs_own_setattr__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_add_shape_to_mask', '_identifier_fields', 'annotation_state_per_task', 'annotations', 'append', 'apply_identifier', 'as_mask', 'created', 'deidentify', 'extend', 'feature_vector', 'filter_annotations', 'filter_by_confidence', 'get_by_shape', 'get_label_names', 'get_labels', 'get_result_media_data', 'has_data', 'has_result_media', 'id', 'kind', 'labels_to_revisit_full_scene', 'map_labels', 'maps', 'media_identifier', 'modified', 'overview', 'prepare_for_post', 'resolve_label_names_and_colors', 'resolve_labels_for_result_media', 'to_dict']

Annotations: [Annotation(labels=[ScoredLabel(probability=0.7790045142173767, name='immature', color='#00ff30ff', id='674b942ae9005102abfe2b1b', source=None)], shape=Polygon(points=[Point(x=1319, y=696), Point(x=1318, y=697), Point(x=1317, y=697), Point(x=1316, y=698), Point(x=1316, y=700), Point(x=1315, y=701), Point(x=1315, y=702), Point(x=1314, y=703), Point(x=1314, y=704), Point(x=1313, y=705), Point(x=1313, y=706), Point(x=1312, y=707), Point(x=1312, y=709), Point(x=1311, y=710), Point(x=1311, y=713), Point(x=1310, y=714), Point(x=1310, y=720), Point(x=1309, y=721), Point(x=1309, y=725), Point(x=1308, y=726), Point(x=1308, y=727), Point(x=1306, y=729), Point(x=1306, y=730), Point(x=1305, y=731), Point(x=1305, y=732), Point(x=1304, y=733), Point(x=1304, y=736), Point(x=1303, y=737), Point(x=1303, y=739), Point(x=1302, y=740), Point(x=1302, y=742), Point(x=1296, y=748), Point(x=1296, y=749), Point(x=1291, y=754), Point(x=1291, y=755), Point(x=1287, y=759), Point(x=1286, y=759), Point(x=1285, y=760), Point(x=1285, y=761), Point(x=1281, y=765), Point(x=1280, y=765), Point(x=1279, y=766), Point(x=1278, y=766), Point(x=1262, y=782), Point(x=1261, y=782), Point(x=1260, y=783), Point(x=1259, y=783), Point(x=1258, y=784), Point(x=1256, y=784), Point(x=1255, y=785), Point(x=1254, y=785), Point(x=1250, y=789), Point(x=1248, y=789), Point(x=1247, y=790), Point(x=1239, y=790), Point(x=1238, y=791), Point(x=1235, y=791), Point(x=1234, y=792), Point(x=1233, y=792), Point(x=1232, y=793), Point(x=1231, y=793), Point(x=1229, y=795), Point(x=1228, y=795), Point(x=1227, y=796), Point(x=1222, y=796), Point(x=1221, y=797), Point(x=1215, y=797), Point(x=1214, y=798), Point(x=1212, y=798), Point(x=1209, y=801), Point(x=1209, y=802), Point(x=1206, y=805), Point(x=1206, y=812), Point(x=1207, y=813), Point(x=1207, y=814), Point(x=1208, y=815), Point(x=1208, y=816), Point(x=1209, y=817), Point(x=1209, y=818), Point(x=1210, y=819), Point(x=1210, y=820), Point(x=1211, y=821), Point(x=1211, y=822), Point(x=1213, y=824), Point(x=1213, y=825), Point(x=1216, y=828), Point(x=1216, y=829), Point(x=1217, y=830), Point(x=1217, y=831), Point(x=1218, y=832), Point(x=1218, y=833), Point(x=1219, y=833), Point(x=1222, y=836), Point(x=1222, y=837), Point(x=1224, y=839), Point(x=1224, y=840), Point(x=1225, y=840), Point(x=1229, y=844), Point(x=1229, y=845), Point(x=1231, y=847), Point(x=1232, y=847), Point(x=1235, y=850), Point(x=1235, y=851), Point(x=1236, y=852), Point(x=1237, y=852), Point(x=1238, y=853), Point(x=1239, y=853), Point(x=1242, y=856), Point(x=1242, y=857), Point(x=1243, y=858), Point(x=1244, y=858), Point(x=1245, y=859), Point(x=1246, y=859), Point(x=1247, y=860), Point(x=1249, y=860), Point(x=1253, y=864), Point(x=1254, y=864), Point(x=1255, y=865), Point(x=1257, y=865), Point(x=1258, y=866), Point(x=1259, y=866), Point(x=1260, y=867), Point(x=1261, y=867), Point(x=1262, y=868), Point(x=1263, y=868), Point(x=1265, y=870), Point(x=1266, y=870), Point(x=1267, y=871), Point(x=1270, y=871), Point(x=1271, y=872), Point(x=1294, y=872), Point(x=1295, y=871), Point(x=1301, y=871), Point(x=1302, y=870), Point(x=1303, y=870), Point(x=1306, y=867), Point(x=1307, y=867), Point(x=1308, y=866), Point(x=1311, y=866), Point(x=1312, y=865), Point(x=1315, y=865), Point(x=1316, y=864), Point(x=1317, y=864), Point(x=1318, y=863), Point(x=1319, y=863), Point(x=1322, y=860), Point(x=1323, y=860), Point(x=1324, y=859), Point(x=1328, y=859), Point(x=1329, y=858), Point(x=1331, y=858), Point(x=1336, y=853), Point(x=1341, y=853), Point(x=1342, y=852), Point(x=1345, y=852), Point(x=1349, y=848), Point(x=1350, y=848), Point(x=1351, y=847), Point(x=1354, y=847), Point(x=1355, y=846), Point(x=1357, y=846), Point(x=1364, y=839), Point(x=1364, y=838), Point(x=1366, y=836), Point(x=1366, y=835), Point(x=1367, y=834), Point(x=1368, y=834), Point(x=1370, y=832), Point(x=1370, y=831), Point(x=1371, y=830), Point(x=1371, y=827), Point(x=1372, y=826), Point(x=1372, y=825), Point(x=1374, y=823), Point(x=1374, y=822), Point(x=1376, y=820), Point(x=1376, y=816), Point(x=1377, y=815), Point(x=1377, y=765), Point(x=1376, y=764), Point(x=1376, y=759), Point(x=1375, y=758), Point(x=1375, y=756), Point(x=1373, y=754), Point(x=1373, y=753), Point(x=1372, y=752), Point(x=1372, y=750), Point(x=1371, y=749), Point(x=1371, y=744), Point(x=1370, y=743), Point(x=1370, y=740), Point(x=1369, y=739), Point(x=1369, y=738), Point(x=1366, y=735), Point(x=1366, y=734), Point(x=1365, y=733), Point(x=1365, y=731), Point(x=1364, y=730), Point(x=1364, y=728), Point(x=1363, y=727), Point(x=1363, y=726), Point(x=1361, y=724), Point(x=1361, y=723), Point(x=1359, y=721), Point(x=1359, y=719), Point(x=1358, y=718), Point(x=1358, y=716), Point(x=1353, y=711), Point(x=1353, y=710), Point(x=1352, y=709), Point(x=1351, y=709), Point(x=1350, y=708), Point(x=1349, y=708), Point(x=1348, y=707), Point(x=1347, y=707), Point(x=1343, y=703), Point(x=1342, y=703), Point(x=1341, y=702), Point(x=1338, y=702), Point(x=1337, y=701), Point(x=1334, y=701), Point(x=1333, y=700), Point(x=1332, y=700), Point(x=1330, y=698), Point(x=1329, y=698), Point(x=1328, y=697), Point(x=1327, y=697), Point(x=1326, y=696)], type=<ShapeType.POLYGON: 'POLYGON'>), modified=None, id=None, labels_to_revisit=None), Annotation(labels=[ScoredLabel(probability=0.772817075252533, name='immature', color='#00ff30ff', id='674b942ae9005102abfe2b1b', source=None)], shape=Polygon(points=[Point(x=419, y=654), ...

## 3. Generating instance segmentation masks from polygons and bounding boxes

This function extracts instance segmentation masks from polygon annotations, combining **detection (bounding boxes)** and **segmentation (masks)** in the same instance using `fo.Detection`.  

1. **Load Image** – Reads and converts the image to RGB.  
2. **Process Annotations** – Extracts polygon points, computes bounding boxes, and normalizes coordinates.  
3. **Generate Masks** – Creates, crops, and resizes binary masks for each annotation.  
4. **Save & Return** – Stores masks as temp files and returns `fo.Detection` objects, ensuring the bounding box and mask belong to the same instance.  

This enables accurate visualization and analysis in FiftyOne, preserving both object localization and shape details.


Useful for visualizing or processing segmentation data in FiftyOne.




In [ ]:
import numpy as np
import cv2
import fiftyone as fo
from PIL import Image as PILImage  # Using PIL's Image
from tempfile import NamedTemporaryFile
import matplotlib.pyplot as plt
from geti_sdk.data_models.shapes import Polygon

# Function to generate an instance segmentation mask from annotation polygons and bounding boxes
def generate_mask_from_polygon_and_bboxes(sample, prediction):
    image = cv2.imread(sample.filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for compatibility

    img_height, img_width = image.shape[:2]  # Get image dimensions
    print(f"Image size: {img_width}x{img_height}")

    detections = []

    for annotation in prediction.annotations:

        # Ensure that the annotation has a shape attribute and that it's a Polygon
        if isinstance(annotation.shape, Polygon):

            # Extract the polygon points from the annotation shape
            polygon_points = [(point.x, point.y) for point in annotation.shape.points]
            polygon_points = np.array(polygon_points, dtype=np.int32)

            # Get the label and confidence of the annotation
            label = annotation.labels[0].name
            confidence = annotation.labels[0].probability

            # Calculate the bounding box from the polygon points
            x, y, w, h = cv2.boundingRect(polygon_points)
            scaled_x = x / img_width
            scaled_y = y / img_height
            scaled_w = w / img_width
            scaled_h = h / img_height
            bounding_box = [scaled_x, scaled_y, scaled_w, scaled_h]

            # Create the mask for the annotation
            mask = np.zeros((img_height, img_width), dtype=np.uint8)
            cv2.fillPoly(mask, [polygon_points], 255)  # Fill the polygon area

            # Crop the mask to fit the bounding box (bounding box coordinates in pixels)
            cropped_mask = mask[y:y + h, x:x + w]

            # Resize the cropped mask to fit the bounding box dimensions
            mask_resized = cv2.resize(cropped_mask, (w, h), interpolation=cv2.INTER_NEAREST)

            # Verify that the resized mask matches the expected size
            print(f"Mask size: {mask_resized.shape} (expected: {h}x{w})")

            # Save the resized mask as a temporary file
            with NamedTemporaryFile(delete=False, suffix='.png') as temp_mask_file:
                mask_path = temp_mask_file.name
                cv2.imwrite(mask_path, mask_resized)  # Save the cropped and resized mask image


            # Create the detection and append it to the detections list
            detection = fo.Detection(
                label=label,
                confidence=confidence,
                bounding_box=bounding_box,  # Scaled bounding box
                mask_path=mask_path  # Path to the mask image
            )
            detections.append(detection)

    return detections


For education purposes check what is happening in the first or last sample. Then you can apply this to the whole dataset

In [ ]:
# Test on one image
sample = dataset.first()  # Get the first sample from the dataset (or pick a specific one)

# Load the image as a NumPy array using PIL or OpenCV
image_path = sample.filepath  # Path to the image file
image_data = PILImage.open(image_path)
image_data = np.array(image_data)  # Convert the image to NumPy array

# Run inference on the sample (using Geti SDK's inference)
prediction = deployment1.infer(image_data)

# Generate the segmentation mask and detections using the annotations from the prediction
detections = generate_mask_from_polygon_and_bboxes(sample, prediction)

# Add the detections as predicted segmentations
sample["predicted_segmentations_test"] = fo.Detections(detections=detections)  # Ensure it's a list of detections

# Save the updated sample
sample.save()

# Reload the dataset to reflect the changes
dataset.reload()

# Print the schema and sample to confirm the changes
print(dataset)

print(sample)

This loop processes each sample in the dataset by loading the image, running inference using Geti SDK, and generating instance segmentation masks. The function extracts detections with both bounding boxes and masks, ensuring they belong to the same instance. These predictions are then stored in the sample under  `"predictions_model1"` or `"predictions_model2"` using `fo.Detections`. Finally, the dataset is reloaded to reflect the updates.


In [ ]:
# Iterate over the samples in the dataset
for sample in dataset:
    # Load the image as a NumPy array using PIL or OpenCV
    image_path = sample.filepath  # Path to the image file
    print(image_path)
    image_data = PILImage.open(image_path)
    image_data = np.array(image_data)  # Convert the image to NumPy array

    # Run inference on the sample (using Geti SDK's inference)
    prediction = deployment2.infer(image_data)

    # Generate the segmentation mask and detections using the annotations from the prediction
    detections = generate_mask_from_polygon_and_bboxes(sample, prediction)

    # Add the detections as predicted segmentations
    sample["predictions_model2"] = fo.Detections(detections=detections)   # Change to  `"predictions_model1"` for evaluation

    # Save the updated sample
    sample.save()

# Reload the dataset to reflect the changes
dataset.reload()

## 4. Model Evaluation


Configuring Plugins in FiftyOne. To use plugins in FiftyOne, we need to enable and configure them properly.
Minimum Configuration Steps:

- Ensure FiftyOne is installed and up to date.
- Download plugind using fiftyone CLI.




In [ ]:
!fiftyone plugins download https://github.com/voxel51/fiftyone-plugins --plugin-names @voxel51/evaluation

## 3. Visualizing the Results

In [ ]:
dataset.reload()
print(dataset)

# Reload the dataset to make sure we get the updated schema
dataset.reload()

# Print the dataset schema
print("Dataset Schema:", dataset.schema)

# Inspect the fields of the first sample to see if 'predicted_segmentations' is added
sample = dataset.first()
print("Sample fields:", sample.field_names)


In [ ]:
session = fo.launch_app(dataset, port=5161, auto=False)

When using the Model Evaluation Plugin in FiftyOne, we can evaluate model predictions using either:
- The Model Evaluation Panel in the FiftyOne App – Interactive UI-based evaluation.
- The FiftyOne SDK – Programmatic evaluation via Python scripts.

👉 Which one to use?

- If you prefer an interactive visual analysis, the FiftyOne App provides an intuitive panel for evaluating models.
- If you need automated and reproducible results, the FiftyOne SDK allows for batch evaluation and detailed reporting.



In [ ]:
eval_key = "model1_eval"

# Now evaluate on the "defect2" field
eval_classif = dataset.evaluate_detections(
    "predictions_model1",
    gt_field="categories_segmentations",
    method="coco", #method is important to see data in the FO app
    classes=["immature", "semimature", "overmature", "mature"],
    eval_key=eval_key,  # <--- store this run under "padim_eval"
)
eval_classif.print_report()


In [ ]:
eval_key_2 = "model2_eval"

# Now evaluate on the "defect2" field
eval_classif = dataset.evaluate_detections(
    "predictions_model2",
    gt_field="categories_segmentations",
    method="coco", #method is important to see data in the FO app
    classes=["immature", "semimature", "overmature", "mature"],
    eval_key=eval_key_2,  # <--- store this run under "padim_eval"
)
eval_classif.print_report()

In [ ]:
new_dataset = dataset.clone()
print(new_dataset)

In [ ]:
export_dir = "coffee_evaluation_FO"
new_dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.FiftyOneDataset,
)
